Finding the orders in raw spectroscopic images is not trivial, especially if the cross-section profiles are funky and/or there are many fibers in a single order. This notebook is a playground for testing how much we can push `numpy` and `scipy` codes in finding the centers and FWHM's of one or more orders in a row of a raw spectral image.

In [1]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
from scipy.ndimage import convolve1d
import ccdproc as cp

%matplotlib notebook

My spectra have flat-topped cross-section profiles, so I will define a box function.

In [ ]:
# Box function
def box(x, l, h):
    # x is the independent variable
    # l is the lower limit of the box
    # h is the upper limit of the box
    if l < x < h:
        return 1.0 + np.random.normal(loc=0., scale=0.001)
    else:
        return 0.0 + np.random.normal(loc=0., scale=0.001)
    
# Guassian function
def gauss(x, mu, sigma):
    return 1./sigma/np.sqrt(2*np.pi)*np.exp(-0.5*((x-mu)/sigma)**2)

Now we need to define our box limits. My spectra have constant distance between each of the 5 fibers, but the distance between orders is variable. Let's plot 3 simulated orders in a row cut of our spectrum.

In [ ]:
fw = 3 # Fiber width
do = 5 # Distance between fibers
offset = [7, 70, 150] # Offset from zero for orders 1, 2 and 3
F = 5 # Number of fibers

# Order 1
ll_o1 = np.array([offset[0]+(fw+do)*xk for xk in range(F)]) # Lower limits
ul_o1 = np.array([offset[0]+fw+(fw+do)*xk for xk in range(F)]) # Upper limits

# Order 2
ll_o2 = np.array([offset[1]+(fw+do)*xk for xk in range(F)]) # Lower limits
ul_o2 = np.array([offset[1]+fw+(fw+do)*xk for xk in range(F)]) # Upper limits

# Order 3
ll_o3 = np.array([offset[2]+(fw+do)*xk for xk in range(F)]) # Lower limits
ul_o3 = np.array([offset[2]+fw+(fw+do)*xk for xk in range(F)]) # Upper limits

In [ ]:
N = 10000
xs = np.linspace(0,200,N)
ys = np.zeros(N, float)

# Order 1
for k in range(F):
    yk = np.zeros(N, float)
    for i in range(N):
        yk[i] = box(xs[i], ll_o1[k], ul_o1[k])
    ys += yk

# Order 2
for k in range(F):
    yk = np.zeros(N, float)
    for i in range(N):
        yk[i] = box(xs[i], ll_o2[k], ul_o2[k])
    ys += yk

# Order 3
for k in range(F):
    yk = np.zeros(N, float)
    for i in range(N):
        yk[i] = box(xs[i], ll_o3[k], ul_o3[k])
    ys += yk
ys = np.array(ys)

# Now we convolve the profiles with a Gaussian:
gau = np.array([gauss(xk, 100., 0.3) for xk in xs])
ys = convolve1d(ys, gau)

plt.plot(xs,ys)
#plt.ylim(-0.2, 1.2)
plt.xlim(0, 200)
plt.show()

Now, let's test how a peak finding routine behaves when we try to identify the peaks corresponding to the 5 fibers of our 3 orders in our row cut.

In [ ]:
peakind = ss.find_peaks_cwt(ys, widths=np.arange(fw*3,fw*100), min_snr=10)

plt.plot(xs,ys)
for pk in peakind:
    plt.plot(xs[pk], ys[pk], '*', color='r')
#plt.ylim(-0.2, 1.2)
plt.xlim(0, 200)
plt.show()